This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [11]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates
import numpy as np

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix) NAME FOR THE FILE, not the title
GRAPH_NAME = "gdp_contrib"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [15]:
today = dt.date.today()

contrib_df = pd.read_csv(
    ut.get_repo_root() / "code" / "data" / "gdp_contrib.csv")

contrib_df = contrib_df.dropna(axis="columns").rename(columns=
    {"Unnamed: 0": "Year",
    "Unnamed: 1": "Quarter"},
)

contrib_df["Date"] = pd.to_datetime(contrib_df["Year"].astype(str) + contrib_df["Quarter"])


contrib_df = contrib_df.set_index("Date").drop(columns=["Year", "Quarter"])

contrib_df = contrib_df.loc[contrib_df.index >= pd.to_datetime("2024-01-01")]

contrib_df.loc['2025-07-01', 'Real GDP'] = np.nan  # July 1, 2025 Nowcast

contrib_df.tail()

C:\Users\Bobby\AppData\Local\Temp\ipykernel_21052\2530312567.py:11: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Real GDP,Consumption,Nonresidential Investment,Residential Investment,Change in Inventories,Net Exports,Government Spending
Date,,,,,,,
2024-07-01,3.3,2.66,0.48,-0.20,-0.11,-0.41,0.92
2024-10-01,1.9,2.61,-0.51,0.17,-0.91,-0.06,0.57
2025-01-01,-0.6,0.42,1.24,-0.04,2.58,-4.68,-0.17
2025-04-01,3.8,1.68,0.98,-0.21,-3.44,4.83,-0.01
2025-07-01,NaN,2.28,0.58,-0.19,0.37,0.58,0.26


In [101]:
fig = go.Figure()


# All colors with "names"
colors_dict = {
    'orange':  '#CF8B40',
    'blue': '#2A547E' ,
    'dark_gray': '#879396',
    'beige':'#F7CAC9',
    'light_blue': '#7095cf',
    'light_gray': '#D3D3D3',
    'dark_blue_text': '#163855'
}

# Ordered colors for order of trace coloring
custom_colorway = [
    colors_dict['orange'],
    colors_dict['dark_blue_text'],
    colors_dict['light_blue'],
    colors_dict['blue'],
    colors_dict['light_gray'],
    colors_dict['dark_gray']
]

bar_columns = [col for col in contrib_df.columns if col != "Real GDP"]

# Add stacked bars for all columns except 'Real GDP', using the custom color sequence
for i, col in enumerate(bar_columns):
    fig.add_trace(
        go.Bar(
            x=contrib_df.index,
            y=contrib_df[col],
            name=col,
            marker_color=custom_colorway[i % len(custom_colorway)]
        )
    )

# Add Real GDP as a line on top
fig.add_trace(
    go.Scatter(
        x=contrib_df.index,
        y=contrib_df["Real GDP"],
        name="Real GDP",
        mode="lines+markers",
        line=dict(color="black", width=3)
    )
)

# # Add annotation for GDPNow at the final point on the x axis
# fig.add_annotation(
#     x=contrib_df.index[-1],
#     y=4.5,
#     text="GDPNow: 3.9%",
#     showarrow=False,
#     font=dict(color="black", size=11),
#     xanchor="center",
#     yanchor="bottom"
# )

fig.update_layout(
    legend=dict(
        font=dict(size= 18),
        orientation="v",
        x=1,
        y=0.5,
        xanchor="left",
        yanchor="middle"
    ),
    barmode='relative',
    title="Real GDP by Component <br><sup>Quarterly, Annualized, Seasonally Adjusted</sup>",
    xaxis_title="Date",
    yaxis_title="Contribution to Percent Change",
    legend_title="Component"
)

fig.add_trace(go.Scatter(
    x=["2025-04-01", "2025-07-01", "2025-10-01"],
    y=[3.8, 2.3, 2.3],
    mode='lines+markers+text',
    text=['','', 'Nowcast: 2.3%'],  # Labels for the dots
    textposition='top center',
    line=dict(dash='dash'),
    textfont=dict(color='green', size=12),
    marker=dict(size=8, color='green', symbol='square'),
    name='NY Fed Nowcast'
))

fig.add_trace(go.Scatter(
    x=["2025-04-01", "2025-07-01"],
    y=[3.8, 3.9],
    mode='lines+markers+text',
    text=['', 'GDPNow: 3.9%'],  # Labels for the dots
    textposition='top center',
    line=dict(dash='dash'),
    textfont=dict(color='purple', size=12),
    marker=dict(size=8, color='purple', symbol='diamond'),
    name='GDPNow'
))


fig.add_trace(go.Scatter(
    x=["2025-01-01", "2025-04-01", "2025-07-01", "2025-10-01"],
    y=[1.6, 1.6, 1.6, 1.6],
    mode='lines+text+markers',
    text=['', '', '', 'SEP: 1.6%'],  # Labels for the dots
    textposition='bottom center',
    line=dict(dash='dash', color='red'),
    textfont=dict(color='red', size=12),
    marker=dict(size=8, color='red', symbol='triangle-up'),
    name='Sep. SEP EOY 2025'
))

fig.update_xaxes(
    dtick="M12",
    tickformat="%Y",
    title_text=None,
)

fig.update_layout(
    height=550,
    width=1150,
)
fig.update_yaxes(ticksuffix="%", tickformat=".f")


fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")
fig.show()